In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re

import pandas as pd

In [2]:
options = Options()
options.add_argument('--headless') 
service = Service(executable_path="./chromedriver")
driver = webdriver.Chrome(service=service, options=options)

In [3]:
def get_data_by_url(url):
    driver.get(url)
    driver.implicitly_wait(60)
    html_content = driver.page_source
    data = BeautifulSoup(html_content, 'html.parser').decode("utf-8")
    return data

In [4]:
main_url = "https://abit-poisk.org.ua/rate2023"
main_data = get_data_by_url(main_url)

In [5]:
regions = re.findall(r'rate2023/\S+', main_data)
regions = [''.join(string.split('">')) for string in regions]

In [6]:
names = []
english_tests = []
tznk_tests = []
mean_scores = []
student_regions = []
student_universities = []
student_faculties = []
student_majors = []
student_statuses = []
student_forms = []

#placeholder = True
#if placeholder == True:
for region_index in range(len(regions)):
    region_url = "https://abit-poisk.org.ua/"+regions[region_index]
    #region_url = "https://abit-poisk.org.ua/"+regions[17]
    
    universities_data = get_data_by_url(region_url)
    univer1 = universities_data.partition('Коледжі та Училища')[0]
    univer2 = universities_data.partition('Відокремлені підрозділи')[2].partition('Інші заклади')[0]
    universities_data = univer1+univer2
    
    student_region = universities_data.partition('class="headline margin-0">\n          ')[2].partition('\n')[0]

    universities = re.findall(r'rate2023/univer/\S+', universities_data)
    universities = [''.join(string.split('">')) for string in universities]
    
    #print(universities)

    for university in universities:
        university_url = "https://abit-poisk.org.ua/"+university
        university_data = get_data_by_url(university_url)
        
        student_university = university_data.partition('class="headline" style="display: inline-block" title="')[2].partition('">\n')[0]
        
        if student_university == '':
            student_university = university_data.partition('class="headline" style="display: inline-block" title=\'')[2].partition('\'>\n')[0]
        
        specialties = re.findall(r'rate2023/direction/\S+', university_data)
        specialties = [''.join(string.split('">')) for string in specialties]

        for i in range(len(specialties)-1,-1,-1):
            statements_url = "https://abit-poisk.org.ua/"+specialties[i]
            statements_data = get_data_by_url(statements_url)

            if statements_data.find("Магістр (на основі: Бакалавр)") != -1:
                faculty = statements_data.partition('<div class="subhead-2 horizontal-scroll-xs">\n        <span title="')[2].partition('">\n')[0] 
                
                if faculty == '':
                    faculty = statements_data.partition('<div class="subhead-2 horizontal-scroll-xs">\n        <span title=\'')[2].partition('\'>\n')[0]
                
                major = statements_data.partition('<h2 class="headline margin-0">\n        ')[2].partition('\n')[0]
                
                form = "заочна"
                if statements_data.find("денна форма") != -1: form = "денна"
                
                statements_data = statements_data.split('<a href="/#search')[1::]

                for statement in statements_data:
                    status = statement.partition('data-header="Статус">\n          ')[2].partition('\n')[0]
                    
                    if status == 'До наказу (б)' or status == 'До наказу (к)':
                        name = statement.partition('target="_blank">\n           ')[2].partition('\n')[0]
                        
                        english = statement.partition('Іноземна мова')[2].partition('<strong>\n              ')[2].partition('\n')[0]
                        tznk = statement.partition('ТЗНК:')[2].partition('<strong>\n              ')[2].partition('\n')[0]
                        
                        if english == '':
                            english = english.replace('','0')
                        if tznk == '':
                            tznk = tznk.replace('','0')
                        
                        mean_score = (float(english)+float(tznk))/2
                        
                        if english != '0' and tznk != '0':
                            names.append(name)
                            english_tests.append(english)
                            tznk_tests.append(tznk)
                            mean_scores.append(mean_score)
                            student_regions.append(student_region)
                            student_universities.append(student_university)
                            student_faculties.append(faculty)
                            student_majors.append(major)
                            student_statuses.append(status)
                            student_forms.append(form)
            
            elif statements_data.find("Молодший бакалавр (на основі: Повна загальна середня освіта)") != -1:
                continue
            else:
                break
        
    students = pd.DataFrame({
        'name': names,
        'english': english_tests,
        'tznk': tznk_tests,
        'mean_score': mean_scores,
        'region': student_regions,
        'university': student_universities,
        'faculty': student_faculties,
        'major': student_majors,
        'status': student_statuses,
        'form': student_forms
    })
    
    students['english'] = students['english'].astype(int)
    students['tznk'] = students['tznk'].astype(int)
    students['mean_score'] = students['mean_score'].astype(float)
    
    students = students.drop_duplicates().reset_index(drop=True)
    
    students.to_csv(f'students_{region_index}.csv', index=False, encoding='utf-8')
    #students.to_csv(f'students_sumy.csv', index=False)